In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

We've talked about Random Forests. Now it's time to build one.

Here we'll use data from Lending Club to predict the state of a loan given some information about it. You can find the dataset [here](https://www.lendingclub.com/info/download-data.action). We'll use 2015 data. ([Thinkful mirror](https://www.dropbox.com/s/m7z42lubaiory33/LoanStats3d.csv?dl=0))

In [2]:
# Replace the path with the correct path for your data.
y2015 = pd.read_csv(
    'LoanStats3d.csv',
    skipinitialspace=True,
    header=1
)

# Note the warning about dtypes.

/Users/maxcalabro/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
y2015.head()

In [ ]:
len(y2015)

## The Blind Approach

Now, as we've seen before, creating a model is the easy part. Let's try just using everything we've got and throwing it without much thought into a Random Forest. SKLearn requires the independent variables to be be numeric, and all we want is dummy variables so let's use `get_dummies` from Pandas and see what happens off of this kind of naive approach.

In [ ]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)

cross_val_score(rfc, X, Y, cv=5)

Did your kernel die? My kernel died.

Guess it isn't always going to be that easy...

Can you think of what went wrong?

(You're going to have to reset your kernel and reload the column, BUT DON'T RUN THE MODEL AGAIN OR YOU'LL CRASH THE KERNEL AGAIN!)

## Data Cleaning

Well, `get_dummies` can be a very memory intensive thing, particularly if data are typed poorly. We got a warning about that earlier. Mixed data types get converted to objects, and that could create huge problems. Our dataset is about 400,000 rows. If there's a bad type there its going to see 400,000 distinct values and try to create dummies for all of them. That's bad. Lets look at all our categorical variables and see how many distinct counts there are...

In [3]:
categorical = y2015.select_dtypes(include=['object'])
for i in categorical:
    column = categorical[i]
    print(i)
    print(column.nunique())

id
421097
term
2
int_rate
110
grade
7
sub_grade
35
emp_title
120812
emp_length
12
home_ownership
4
verification_status
3
issue_d
12
loan_status
7
pymnt_plan
1
url
421095
desc
34
purpose
14
title
27
zip_code
914
addr_state
49
earliest_cr_line
668
revol_util
1211
initial_list_status
2
last_pymnt_d
25
next_pymnt_d
4
last_credit_pull_d
26
application_type
2
verification_status_joint
3


Well that right there is what's called a problem. Some of these have over a hundred thousand distinct types. Lets drop the ones with over 30 unique values, converting to numeric where it makes sense. In doing this there's a lot of code that gets written to just see if the numeric conversion makes sense. It's a manual process that we'll abstract away and just include the conversion.

You could extract numeric features from the dates, but here we'll just drop them. There's a lot of data, it shouldn't be a huge problem.

In [4]:
# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc'], 1, inplace=True)

Wonder what was causing the dtype error on the id column, which _should_ have all been integers? Let's look at the end of the file.

In [ ]:
y2015.tail()

In [5]:
# Remove two summary rows at the end that don't actually contain data.
y2015 = y2015[:-2]

Now this should be better. Let's try again.

In [ ]:
pd.get_dummies(y2015)

It finally works! We had to sacrifice sub grade, state address and description, but that's fine. If you want to include them you could run the dummies independently and then append them back to the dataframe.

## Second Attempt

Now let's try this model again.

We're also going to drop NA columns, rather than impute, because our data is rich enough that we can probably get away with it.

This model may take a few minutes to run.

In [7]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)



In [ ]:
cross_val_score(rfc, X, Y, cv=10)

The score cross validation reports is the accuracy of the tree. Here we're about 98% accurate.

That works pretty well, but there are a few potential problems. Firstly, we didn't really do much in the way of feature selection or model refinement. As such there are a lot of features in there that we don't really need. Some of them are actually quite impressively useless.

There's also some variance in the scores. The fact that one gave us only 93% accuracy while others gave higher than 98 is concerning. This variance could be corrected by increasing the number of estimators. That will make it take even longer to run, however, and it is already quite slow.

## DRILL: Third Attempt

So here's your task. Get rid of as much data as possible without dropping below an average of 90% accuracy in a 10-fold cross validation.

You'll want to do a few things in this process. First, dive into the data that we have and see which features are most important. This can be the raw features or the generated dummies. You may want to use PCA or correlation matrices.

Can you do it without using anything related to payment amount or outstanding principal? How do you know?

In [8]:
corrmat = X.copy()

In [9]:
corr_frac = corrmat.sample(frac=.02).corr()

In [10]:
corr_frac.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,...,last_credit_pull_d_Nov-2016,last_credit_pull_d_Oct-2015,last_credit_pull_d_Oct-2016,last_credit_pull_d_Sep-2015,last_credit_pull_d_Sep-2016,application_type_INDIVIDUAL,application_type_JOINT,verification_status_joint_Not Verified,verification_status_joint_Source Verified,verification_status_joint_Verified
id,1.000000,0.998797,-0.012244,-0.012244,-0.012517,-0.060417,-0.018473,-0.000723,0.014566,-0.000913,...,-0.021035,-0.016947,-0.080718,-0.032915,-0.007227,-0.041079,0.041079,0.031287,NaN,0.026791
member_id,0.998797,1.000000,-0.011956,-0.011956,-0.012228,-0.060822,-0.017985,-0.000284,0.013992,-0.000786,...,-0.020688,-0.017055,-0.080442,-0.032882,-0.007275,-0.041347,0.041347,0.031460,NaN,0.026996
loan_amnt,-0.012244,-0.011956,1.000000,1.000000,0.999994,0.144798,0.942555,0.403188,0.016611,-0.038436,...,0.010823,0.016994,0.018812,0.007634,-0.004603,-0.035878,0.035878,0.007625,NaN,0.043100
funded_amnt,-0.012244,-0.011956,1.000000,1.000000,0.999994,0.144798,0.942555,0.403188,0.016611,-0.038436,...,0.010823,0.016994,0.018812,0.007634,-0.004603,-0.035878,0.035878,0.007625,NaN,0.043100
funded_amnt_inv,-0.012517,-0.012228,0.999994,0.999994,1.000000,0.144463,0.942541,0.403280,0.016448,-0.038474,...,0.010859,0.017003,0.018756,0.007669,-0.004571,-0.035787,0.035787,0.007631,NaN,0.042964


A lot of these columns are highly correlated to each other, but I'm not sure how to pick out the worst offenders... Let's first get rid of more of the columns with high number of possible outcomes.

In [11]:
y2015.drop(['title', 'last_credit_pull_d'], 1, inplace=True)

In [12]:
categorical = y2015.select_dtypes(include=['object'])
for i in categorical:
    column = categorical[i]
    print(i)
    print(column.nunique())

term
2
grade
7
emp_length
12
home_ownership
4
verification_status
3
issue_d
12
loan_status
7
pymnt_plan
1
purpose
14
initial_list_status
2
last_pymnt_d
25
next_pymnt_d
4
application_type
2
verification_status_joint
3


In [13]:
y2015.drop(['last_pymnt_d'], 1, inplace=True)

In [ ]:
rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

In [14]:
y2015.drop(['issue_d', 'purpose', 'emp_length'], 1, inplace=True)

In [15]:
rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

In [16]:
X.shape

(421095, 86)

In [17]:
X.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,...,initial_list_status_w,next_pymnt_d_Feb-2017,next_pymnt_d_Jan-2017,next_pymnt_d_Jul-2016,next_pymnt_d_Mar-2017,application_type_INDIVIDUAL,application_type_JOINT,verification_status_joint_Not Verified,verification_status_joint_Source Verified,verification_status_joint_Verified
0,68009401.0,72868139.0,16000.0,16000.0,16000.0,14.85,379.39,48000.0,33.18,0.0,...,1,0,1,0,0,1,0,0,0,0
1,68354783.0,73244544.0,9600.0,9600.0,9600.0,7.49,298.58,60000.0,22.44,0.0,...,1,0,1,0,0,1,0,0,0,0
2,68466916.0,73356753.0,25000.0,25000.0,25000.0,7.49,777.55,109000.0,26.02,0.0,...,1,0,0,0,0,1,0,0,0,0
3,68466961.0,73356799.0,28000.0,28000.0,28000.0,6.49,858.05,92000.0,21.60,0.0,...,1,0,1,0,0,1,0,0,0,0
4,68495092.0,73384866.0,8650.0,8650.0,8650.0,19.89,320.99,55000.0,25.49,0.0,...,1,0,0,0,0,1,0,0,0,0


In [18]:
y2015.drop(['id', 'member_id', 'loan_amnt'], 1, inplace=True)

In [20]:
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

In [21]:
X.shape

(421095, 83)

In [ ]:
cross_val_score(rfc, X, Y, cv=10)

In [22]:
y2015.shape

(421095, 94)

In [23]:
y2015.columns

Index(['funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment',
       'grade', 'home_ownership', 'annual_inc', 'verification_status',
       'loan_status', 'pymnt_plan', 'dti', 'delinq_2yrs', 'inq_last_6mths',
       'mths_since_last_delinq', 'mths_since_last_record', 'open_acc',
       'pub_rec', 'revol_bal', 'total_acc', 'initial_list_status', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_amnt', 'next_pymnt_d',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint',
       'verification_status_joint', 'acc_now_delinq', 'tot_coll_amt',
       'tot_cur_bal', 'open_acc_6m', 'open_il_6m', 'open_il_12m',
       'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util',
       'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util',
       

In [24]:
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']

In [25]:
cols = np.random.choice(len(X.columns), 40, replace=False)
cols

array([89, 32, 85, 21, 12, 82, 31,  2, 42, 52, 27, 62,  3, 87, 17, 79, 47,
       37,  5,  1, 80,  4, 84, 15, 74, 70, 72, 28, 24, 50, 40, 23, 73, 39,
       36, 16, 91,  7, 81,  9])

In [26]:
X.drop(X.columns[cols],axis=1,inplace=True)

In [27]:
X.shape

(421095, 53)

In [28]:
X = pd.get_dummies(X)
X = X.dropna(axis=1)

Trying to just remove a whole bunch of variables at random.

In [29]:
X.shape

(421095, 42)

In [30]:
cross_val_score(rfc, X, Y, cv=10)

array([ 0.9606991 ,  0.95991546,  0.95996296,  0.95979673,  0.95946331,
        0.95943956,  0.95993731,  0.95891615,  0.95915168,  0.95855698])

Well, dropping 40 rows didn't decrease our accuracy.

In [33]:
X_simple = X.loc[:,['funded_amnt', 'total_pymnt', 'last_pymnt_amnt', 'delinq_amnt']]

In [35]:
cross_val_score(rfc, X_simple, Y, cv=10)

array([ 0.72543991,  0.9141317 ,  0.8814799 ,  0.91199449,  0.90945144,
        0.89724531,  0.8693391 ,  0.89351445,  0.90737882,  0.9112478 ])

Finally got below 90! Let's add a few more colums and see if we can recover some of that accuracy.

In [36]:
X.columns

Index(['funded_amnt', 'dti', 'delinq_2yrs', 'total_acc', 'out_prncp',
       'total_pymnt', 'total_rec_int', 'total_rec_late_fee', 'last_pymnt_amnt',
       'policy_code', 'acc_now_delinq', 'acc_open_past_24mths', 'avg_cur_bal',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_rev_tl_op',
       'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc',
       'num_accts_ever_120_pd', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl',
       'num_tl_90g_dpd_24m', 'tax_liens', 'total_bal_ex_mort',
       'total_il_high_credit_limit', 'home_ownership_ANY',
       'home_ownership_MORTGAGE', 'home_ownership_OWN', 'home_ownership_RENT',
       'verification_status_Not Verified',
       'verification_status_Source Verified', 'verification_status_Verified',
       'initial_list_status_f', 'initial_list_status_w',
       'next_pymnt_d_Feb-2017', 'next_pymnt_d_Jan-2017',
       'next_pymnt_d_Jul-2016', 'next_pymnt_d_Mar-2017',
       'application_type_INDIVIDUAL', 'application_type_JOINT'],
  

In [43]:
X_simple = y2015.loc[:,['funded_amnt', 'total_pymnt', 'last_pymnt_amnt', 'delinq_amnt', 'home_ownership', 'verification_status', 'mths_since_last_delinq']]

X_simple = pd.get_dummies(X_simple)
X_simple = X_simple.dropna(axis=1)

In [44]:
cross_val_score(rfc, X_simple, Y, cv=10)

array([ 0.73534231,  0.92201563,  0.9118995 ,  0.91199449,  0.90771788,
        0.90465448,  0.89721912,  0.90629082,  0.91319733,  0.9084216 ])

That's right around around 90% accuracy, although the .73 group suggests we're missing something. But we only have 7 of the original variables, which is pretty cool.

In [45]:
X_simple.shape

(421095, 11)